## 5-Minute Quick Demo of Diaspora Action Provider Features

In [ ]:
"""Import necessary libraries for Demo."""
from __future__ import annotations

import json
import time
import uuid

import globus_sdk
import globus_sdk.scopes
from diaspora_event_sdk import Client
from diaspora_event_sdk.version import __version__ as diaspora_sdk_version
from globus_sdk import __version__ as globus_sdk_version

In [ ]:
# c = Client() # re-auth if there's something wrong with the access token
# c.logout()

In [ ]:
c = Client()
print('Globus SDK version:', globus_sdk_version)
print('Diaspora Event SDK version:', diaspora_sdk_version)
print("User's OpenID:", c.subject_openid)

In [ ]:
# ID of this tutorial notebook as registered with Globus Auth
CLIENT_ID = 'f794186b-f330-4595-b6c6-9c9d3e903e47'

# Do a native app authentication flow to get tokens that allow us
# to interact with the Globus Flows service

scopes = [
    'openid',
    'profile',
    'email',
    globus_sdk.FlowsClient.scopes.manage_flows,
    globus_sdk.FlowsClient.scopes.run_manage,
]
native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
native_auth_client.oauth2_start_flow(requested_scopes=scopes)
print(f'Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}')

auth_code = input('Authorization Code: ')
response = native_auth_client.oauth2_exchange_code_for_tokens(auth_code)

tokens = response.by_resource_server
print(json.dumps(tokens, indent=2))

flows_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens['flows.globus.org']['access_token'],
)
flows_client = globus_sdk.FlowsClient(authorizer=flows_authorizer)

In [ ]:
topic1 = 'topic1-' + c.subject_openid[-12:]
print('Topic name:', topic1)
print(c.register_topic(topic1))

In [ ]:
# Create an Auth client so we can look up identities
auth_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens['auth.globus.org']['access_token'],
)
ac = globus_sdk.AuthClient(authorizer=auth_authorizer)

# Get the user's primary identity
primary_identity = ac.oauth2_userinfo()
identity_id = primary_identity['sub']

print(f"Username: {primary_identity['preferred_username']}")
print(f'ID: {identity_id}')
print('Topic to produce/consume:', topic1)

### 1. Produce events to AP with event keys

In [ ]:
action_url = 'https://diaspora-action-provider.ml22sevubfnks.us-east-1.cs.amazonlightsail.com/'

flow_definition = {
    'Comment': 'Publish messages to Diaspora Event Fabric',
    'StartAt': 'PublishMessages',
    'States': {
        'PublishMessages': {
            'Comment': 'Send messages to a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'msgs.$': '$.input.msgs',
                'keys.$': '$.input.keys',
            },
            'ResultPath': '$.PublishMessages',
            'End': True,
        },
    },
}

In [ ]:
flow_title = f'Diapora-AP-Flow-{str(uuid.uuid4())[:4]}'
flow = flows_client.create_flow(
    title=flow_title,
    definition=flow_definition,
    input_schema={},
)
flow_id = flow['id']
flow_scope = globus_sdk.SpecificFlowClient(flow_id).scopes.make_mutable('user')
print(f"Successfully created flow: '{flow_title} (ID: {flow_id})")
print(f'View the flow in the Web App: https://app.globus.org/flows/{flow_id}')

if flow_id not in tokens:
    # Do a native app authentication flow and get tokens that
    # include the newly deployed flow scope
    native_auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
    native_auth_client.oauth2_start_flow(requested_scopes=flow_scope)
    print(f'Login Here:\n\n{native_auth_client.oauth2_get_authorize_url()}')

    # Authenticate and come back with your authorization code;
    # paste it into the prompt below.
    auth_code = input('Authorization Code: ')
    token_response = native_auth_client.oauth2_exchange_code_for_tokens(
        auth_code,
    )

    # Save the new token in a place where the flows client can retrieve it.
    tokens[flow_id] = token_response.by_resource_server[flow_id]

    # These are the saved scopes for the flow
    print(json.dumps(tokens, indent=2))

In [ ]:
flow_input = {
    'input': {
        'action': 'produce',
        'topic': topic1,
        'msgs': [
            {'content1': 'argonne national laboratory'},
            {'content2': 'university of chicago'},
            {'content3': 'johns hopkins university'},
        ],
        'keys': [
            'my-key-123',
            'my-key-456',
            'my-key-789',
        ],
    },
}

In [ ]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))

### 2. Blocking consume events from AP with specific prefix after a timestamp

In [ ]:
flow_definition_b = {
    'Comment': 'Consume messages to Diaspora Event Fabric',
    'StartAt': 'ConsumeMessages',
    'States': {
        'ConsumeMessages': {
            'Comment': 'Receive messages from a specified topic in Diaspora',
            'Type': 'Action',
            'ActionUrl': action_url,
            'Parameters': {
                'action.$': '$.input.action',
                'topic.$': '$.input.topic',
                'ts.$': '$.input.ts',
                # 'group_id.$': '$.input.group_id',
                'filters.$': '$.input.filters',
            },
            'ResultPath': '$.ConsumeMessages',
            'End': True,
        },
    },
}

flows_client.update_flow(
    flow_id,
    definition=flow_definition_b,
    input_schema={},
)

In [ ]:
ts_now = int(time.time() * 1000)
group_id = 'group-abc'
print(ts_now)
# print(group_id)

flow_input_b = {
    'input': {
        'action': 'consume',
        'topic': topic1,
        'ts': ts_now,
        # 'group_id': group_id,
        'filters': [
            {'Pattern': {'value': {'content': [{'prefix': 'university'}]}}},
            {'Pattern': {'value': {'content': [{'suffix': 'university'}]}}},
        ],
    },
}

In [ ]:
# Get a client for the flow
specific_flow_authorizer = globus_sdk.AccessTokenAuthorizer(
    access_token=tokens[flow_id]['access_token'],
)
print(tokens[flow_id]['access_token'])
specific_flow_client = globus_sdk.SpecificFlowClient(
    flow_id=flow_id,
    authorizer=specific_flow_authorizer,
)

# Run the flow
# Set a descriptive label for this flow run
run_label = f"Diaspora AP Flow by {primary_identity['preferred_username']}"
run = specific_flow_client.run_flow(
    body=flow_input_b,
    label=run_label,
    tags=['tutorial', 'diaspora'],
)

# Get run details
run_id = run['run_id']
run_status = run['status']
print('This flow can be monitored in the Web App:')
print(f'https://app.globus.org/runs/{run_id}')
print(f'Flow run started with ID: {run_id} - Status: {run_status}')

# Poll the Flow service to check on the status of the flow
while run_status == 'ACTIVE':
    time.sleep(5)
    run = flows_client.get_run(run_id)
    run_status = run['status']
    print(f'Run status: {run_status}')

# Run completed
print(json.dumps(run.data, indent=2))